In [4]:
import pandas as pd
device_df = pd.read_csv('device.csv')
logon_df = pd.read_csv('logon.csv')
psycho_df = pd.read_csv('psychometric.csv', index_col=0)

In [5]:
import math
import random
device_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
device_df["insider"] = device_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(device_df[device_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
device_df = device_df.drop(del_rows)
device_df

,id,date,user,pc,activity,insider
2,{U1V9-Z7XT67KV-5649MYHI},01/02/2010 07:59:11,HPH0075,PC-2417,Connect,1
15,{G8U2-N0YV97HS-0321CNBR},01/02/2010 09:22:55,BQS0525,PC-0269,Connect,1
22,{Z2H1-N5DI43BJ-5382DOVG},01/02/2010 09:44:24,RRC0553,PC-6672,Disconnect,1
27,{R2M5-Y4SH41AL-8396QOYF},01/02/2010 09:51:56,RAW0915,PC-2113,Disconnect,1
38,{L0X7-E5FY66XM-6531GXME},01/02/2010 10:33:23,HPH0075,PC-2417,Disconnect,1
...,...,...,...,...,...,...
405333,{A8K9-J9EK01YL-1970CDSX},05/16/2011 18:10:28,KCO0122,PC-9174,Disconnect,1
405338,{W4V7-U0OK13JU-4386IBNV},05/16/2011 18:27:22,CGB0637,PC-8508,Disconnect,1
405343,{Q3P1-P4RO45PO-8346VYCE},05/16/2011 18:34:16,SAA0999,PC-4559,Connect,1
405345,{U6Y2-J9DD01DS-0481JYUP},05/16/2011 18:45:34,BRM0995,PC-0768,Connect,1


In [6]:
import math
import random
logon_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
logon_df["insider"] = logon_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(logon_df[logon_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
logon_df = logon_df.drop(del_rows)
logon_df

,id,date,user,pc,activity,insider
7,{D0P1-U5IU49QG-5472TMLI},01/02/2010 07:08:00,NOB0181,PC-3446,Logon,1
14,{P3Y3-J4EZ51PP-3037PQQN},01/02/2010 07:37:00,ABC0174,PC-5156,Logon,0
22,{A4W2-E9VG78GM-9142SACC},01/02/2010 07:48:00,IKP0472,PC-3842,Logon,1
27,{G6R4-M3PG67RO-6057NVVO},01/02/2010 07:56:00,ESJ0670,PC-2321,Logon,1
49,{A2A0-L1VB61KD-7527DNFG},01/02/2010 08:54:00,BRB0355,PC-9896,Logon,1
...,...,...,...,...,...,...
854816,{M1U4-L3ER46ER-8016FNOM},05/16/2011 21:21:45,IBB0359,PC-2511,Logoff,1
854828,{B3I1-J2DT69BV-1464VECG},05/16/2011 21:38:56,ACC0042,PC-5066,Logoff,1
854840,{U7D5-A7JQ59GY-2200NTUK},05/16/2011 22:28:47,UIR0043,PC-4127,Logon,1
854841,{I7W7-V1ZN51SV-5812FQNK},05/16/2011 22:30:38,UIR0043,PC-4127,Logoff,1


In [7]:
device_df["activity"] = device_df["activity"].apply(lambda x : 0 if x == "Connect" else  1) # activityを0, 1に変換
device_df["date"] = pd.to_datetime(device_df["date"]) # 日時をdatetime型に変換

In [8]:
logon_df["activity"] = logon_df["activity"].apply(lambda x : 0 if x == "Logon" else  1) # activityを0, 1に変換
logon_df["date"] = pd.to_datetime(logon_df["date"]) # 日時をdatetime型に変換

In [9]:
def get_timezone(hour: int) -> int:
        if 0 <= hour and hour < 6:
            return 0
        elif 6 <= hour and hour < 12:
            return 1
        elif 12 <= hour and hour < 18:
            return 2
        else:
            return 3

users = device_df['user'].unique()
d = {}
for user in users:
    d[user] = {} # connect, disconnectを記録
    for index, row in device_df[device_df['user']== user].iterrows():
        ym = str(row['date'].year)+str(row['date'].month)
        hour = row['date'].hour
        if(row['activity'] == 0):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz] += 1 
        if(row['activity'] == 1):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz+4] += 1 

users = logon_df['user'].unique()
for user in users:
    if user not in d:
        d[user] = {} # logon, logoffを記録
    for index, row in logon_df[logon_df['user']== user].iterrows():
        ym = str(row['date'].year)+str(row['date'].month)
        hour = row['date'].hour
        if(row['activity'] == 0):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz+8] += 1 
        if(row['activity'] == 1):
            if ym not in d[user]:
                d[user][ym] = [0]*16
            tz = get_timezone(hour)
            d[user][ym][tz+12] += 1 

In [17]:
# USBの挿抜に関する列を追加

main_df = pd.DataFrame({'user':[], 'O': [], 'C': [], 'E': [], 'A': [], 'N': [], 
                        'dev_con_0to6': [], 'dev_con_6to12': [], 'dev_con_12to18': [], 'dev_con_18to24': [],
                        'dev_dis_0to6': [], 'dev_dis_6to12': [], 'dev_dis_12to18': [], 'dev_dis_18to24': [], 
                        'pc_on_0to6': [], 'pc_on_6to12': [], 'pc_on_12to18': [], 'pc_on_18to24': [],
                        'pc_off_0to6': [], 'pc_off_6to12': [], 'pc_off_12to18': [], 'pc_off_18to24': [],
                       }, index=[])

for user, v in d.items():
    for ym, vv in v.items():
        row = psycho_df[psycho_df['user_id']==user]
        main_df = main_df.append({'user': user, 'O': row['O'][0], 'C':  row['C'][0], 'E':  row['E'][0], 'A':  row['A'][0], 'N':  row['N'][0],  
                        'dev_con_0to6':  vv[0], 'dev_con_6to12': vv[1], 'dev_con_12to18': vv[2], 'dev_con_18to24': vv[3],
                        'dev_dis_0to6': vv[4], 'dev_dis_6to12': vv[5], 'dev_dis_12to18': vv[6], 'dev_dis_18to24': vv[7], 
                        'pc_on_0to6': vv[8], 'pc_on_6to12': vv[9], 'pc_on_12to18': vv[10], 'pc_on_18to24': vv[11],
                        'pc_off_0to6': vv[12], 'pc_off_6to12': vv[13], 'pc_off_12to18': vv[14], 'pc_off_18to24': vv[15],
                       }, ignore_index=True)
main_df

,user,O,C,E,A,N,dev_con_0to6,dev_con_6to12,dev_con_12to18,dev_con_18to24,...,dev_dis_12to18,dev_dis_18to24,pc_on_0to6,pc_on_6to12,pc_on_12to18,pc_on_18to24,pc_off_0to6,pc_off_6to12,pc_off_12to18,pc_off_18to24
0,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,13.0,13.0,0.0,...,11.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,3.0,0.0
1,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,12.0,10.0,0.0,...,18.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0,3.0,0.0
2,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,9.0,13.0,0.0,...,7.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,2.0,0.0
3,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,10.0,17.0,0.0,...,11.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0
4,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,18.0,14.0,0.0,...,15.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15835,RGN0408,11.0,21.0,22.0,22.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0
15836,RGN0408,11.0,21.0,22.0,22.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
15837,RGN0408,11.0,21.0,22.0,22.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0
15838,RGN0408,11.0,21.0,22.0,22.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
main_df.assign(insider=0) # insiderのラベル追加
insiders_df = pd.read_csv('insiders.csv')
insiders = insiders_df[insiders_df['dataset'] == 4.2]['user'].tolist()
main_df["insider"] = main_df["user"].apply(lambda x : 0 if x  in insiders else  1)
nin_insider_list = list(main_df[main_df['insider'] == 1].index) 
del_rows = random.sample(nin_insider_list, math.floor(0.87*len(nin_insider_list))) # insiderとnot insiderの割合をいい感じにするためにnot insiderを削除
main_df = main_df.drop(del_rows)
main_df

,user,O,C,E,A,N,dev_con_0to6,dev_con_6to12,dev_con_12to18,dev_con_18to24,...,dev_dis_18to24,pc_on_0to6,pc_on_6to12,pc_on_12to18,pc_on_18to24,pc_off_0to6,pc_off_6to12,pc_off_12to18,pc_off_18to24,insider
0,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,13.0,13.0,0.0,...,0.0,0.0,5.0,1.0,0.0,0.0,0.0,3.0,0.0,1
1,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,12.0,10.0,0.0,...,0.0,0.0,7.0,2.0,0.0,0.0,0.0,3.0,0.0,1
3,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,10.0,17.0,0.0,...,0.0,0.0,6.0,6.0,0.0,0.0,0.0,1.0,0.0,1
4,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,18.0,14.0,0.0,...,0.0,0.0,3.0,2.0,0.0,0.0,0.0,4.0,0.0,1
5,HPH0075,40.0,42.0,48.0,25.0,28.0,0.0,16.0,12.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,4.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15796,TLM0797,43.0,42.0,34.0,23.0,36.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,1
15810,CNM0800,32.0,11.0,40.0,38.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,0.0,1
15813,CNM0800,32.0,11.0,40.0,38.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,1
15831,RGN0408,11.0,21.0,22.0,22.0,32.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1


In [12]:
df_ins = (main_df["insider"] == 0)
print("insider :", df_ins.sum()) # 内部不正者のログ件数をカウント
df_niins = (main_df["insider"] == 1)
print("not insider :", df_niins.sum()) # 内部不正者でない人のログ件数をカウント
print(df_ins.sum()/(df_ins.sum()+df_niins.sum())) # 全体における内部不正者のログ件数の割合

insider : 760
not insider : 1961
0.2793090775450202


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np

X = main_df[['O', 'C', 'E', 'A', 'N', 'dev_con_0to6', 'dev_con_6to12', 'dev_con_12to18', 'dev_con_18to24', 'dev_dis_0to6', 'dev_dis_6to12', 'dev_dis_12to18', 'dev_dis_18to24', 'pc_on_0to6', 'pc_on_6to12', 'pc_on_12to18', 'pc_on_18to24', 'pc_off_0to6', 'pc_off_6to12', 'pc_off_12to18', 'pc_off_18to24']]
Y = main_df['insider']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする

# ロジスティック回帰で学習
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, Y_train)
print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)

Y_pred = lr.predict(X_test)
np.set_printoptions(threshold=np.inf)
print(Y_pred)

coefficient =  [[-0.0062635  -0.08673333  0.0163336   0.03044172 -0.02558553  0.30671473
  -0.00669204  0.02481936 -0.05040785  0.24108657  0.00640587 -0.12711702
  -0.3828393   0.1119355  -0.47029352  0.17901273 -0.12243855  0.5020059
   0.07160137 -0.56815118 -0.61322739]]
intercept =  [11.48945184]
[1 1 0 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1
 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0
 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1
 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1 0 0
 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0
 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1
 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 1
 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 1
 1 0 0 1 1 1 1 1 0 0 0

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# モデルの評価
print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

confusion matrix = 
 [[146   6]
 [  1 392]]
accuracy =  0.9871559633027523
precision =  0.9849246231155779
recall =  0.9974554707379135
f1 score =  0.9911504424778761
